In [1]:
# 필요한 라이브러리 import
import pymysql
import pandas as pd
import random

# 데이터베이스 연결
db = pymysql.connect(user='hj', password='1234', host='112.161.7.178', db='movie_db', cursorclass=pymysql.cursors.DictCursor)
cursor = db.cursor()

# 배우 테이블 조회
query = "SELECT * FROM 감독"
cursor.execute(query)
director_data = cursor.fetchall()

# 데이터프레임 생성
db_movie_df = pd.DataFrame(director_data)

In [2]:
# filmoNames에 데이터를 분리, 랜덤 출력
def separated_string(string):
    choices = string.split("|")
    randomMovieName = random.choice(choices)
    return randomMovieName

In [3]:
# 감독이 한명일 때 실행될 함수
def onePeople(director, df):

    director_movies = df[df['peopleNm'] == director]['filmoNames'].tolist()
    # 검색 감독이 제작한 영화가 없으면 None 리턴
    if not director_movies:
        return None
    else:
        # 랜덤으로 추천
        recommended_movie = separated_string(random.choice(director_movies))
        return recommended_movie

In [4]:
# 한명이 아닐 때 실행될 함수
def twoPeople(user_input_director, df, movie_name):
    
    # 새로운 조건으로 데이터 검색
    query = "SELECT * FROM 감독 WHERE peopleNm = %s AND filmoNames LIKE %s"
    cursor.execute(query, (user_input_director, f'%{movie_name}%'))
    twoPeopleDirectorData = cursor.fetchall()

    # 새로운 데이터프레임 생성
    df = pd.DataFrame(twoPeopleDirectorData)
    director_movies = df[df['peopleNm'] == user_input_director]['filmoNames'].tolist()
    
    if not director_movies:
        return None
    else:
        recommended_movie = separated_string(random.choice(director_movies))
        return recommended_movie

In [5]:
# 사용자 입력 받기
user_input_director = input("감독의 이름을 입력하세요: ").strip()

In [6]:
# 검색 결과 출력
search_result = db_movie_df[db_movie_df['peopleNm'] == user_input_director]

if not search_result.empty:

    # 입력받은 감독이 한명일 때
    if len(search_result) == 1:
        recommended_movie = onePeople(user_input_director, db_movie_df)
    
        # 결과 출력
        if recommended_movie:
            print("추천 영화:", recommended_movie)
        else:
            print(f"이름이 {user_input_director}인 감독이 제작한 영화가 없습니다.")

    # 입력받은 감독이 한명이 아닐 때 해당 감독 제작 영화명 입력받기
    else:
        movie_name = input("영화명을 입력하세요: ").strip()
        recommended_movie = twoPeople(user_input_director, db_movie_df, movie_name)

        # 결과 출력
        if recommended_movie:
            print("추천 영화:", recommended_movie)
        else:
            print(f"이름이 {user_input_director}인 감독이 제작한 영화가 없습니다.")
else:
    print(f"'{user_input_director}'으로 검색된 감독이 없습니다.")

추천 영화: 해무(海霧)
